In [6]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
api_key = "e30eb1259555481b91906de4cb5a3cd5"
azure_endpoint = "https://artirst-assistant.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-3.5-turbo-instruct",
    deployment_name="35instruct",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="emb3large",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [8]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [9]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import FlatReader, HTMLTagReader
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("test_data/")

docs = reader.load_data()

In [6]:
print(len(docs))

1


In [7]:
docs

[Document(id_='ece0de1c-27df-4aa7-944f-554e783e794a', embedding=None, metadata={'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 'text/html', 'file_size': 2626, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='<div>\n  <h1 class="c-ttl-article__txt">結婚12年目「倖田來未」が明かす“夫と息子”との私生活\u3000「親子でプロレスにハマってます」</h1>\n  <p>\n    今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆\n    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。\n  </p>\n  <p>\n    倖田來未が1月18

In [8]:
# Indexing documentation 
index = VectorStoreIndex.from_documents(docs)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("倖田來未さんは結婚して何年目ですか？")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
12年目


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is Koda Kumi and how long has she been in the music industry?")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"

Koda Kumi is a 40-year-old singer who has been in the music industry for 23 years.


In [16]:
query_engine = index.as_query_engine()
response = query_engine.query("倖田來未とは何者で、音楽業界でどれくらい働いていますか?")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"

倖田來未は歌手であり、今年でデビュー23年目を迎えています。


---

### Test Part 2: Deeper

In [1]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
api_key = "a83e2f795b9f4aada15fba7f6594d216"
azure_endpoint = "https://artist-assistant-uk-south.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="35turbo16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada2",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [3]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler

from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("datanotclean/")

docs = reader.load_data()


pipeline = IngestionPipeline(
    transformations=[
        HTMLNodeParser.from_defaults(tags=["p", "h1", "br"]),
        SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
        ),
        embed_model,
    ],
)
nodes = pipeline.run(documents=docs)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https:/

In [6]:
print(len(nodes))

4


In [7]:
for node in nodes: 
    print(node)
    print("-----------------------")

Node ID: da0790e3-95c0-4f04-a97c-896560c2989d
Text: 結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
-----------------------
Node ID: 744f5eef-e73e-4883-87e7-62be3a877ed0
Text: 今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロか
っこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆     Live Package「WIN
GS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与
えているかをインタビューした。 倖田來未が1月18日に発売したMusic＆Live     Package「WINGS」のメイントラックとな
るミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作
家名義...
-----------------------
Node ID: a3ea6ce2-da49-469a-8a67-1d01d29cb8b6
Text: アーティスト名:倖田 來未(Koda Kumi) 生年月日: 1982年 11月13日 出身地:京都府
2000年11月「TAKE BACK」にて全米デビュー。 全米ビルボードダンスチャートで最高位18位を記録。
日本では同曲で2000年12月にデビュー。 続くセカンドシングル「Trust Your Love」のremix盤は全米ビルボード総合
チャートでも最高位19位を獲得し、日本人7人目のランクインの快挙となる。
2005年、06年にそれぞれ発売したベストアルバムはいずれも200万枚を突破
2007年12月には東京ドームでの単独公演を開催し、大成功を収めた。 2008年6月発売した「MOON」ではブラック・
アイド・ピーズ“ファーギー”とのコラボレーション曲「That Ain't ...
-----------------------
Node ID: 11970202-85

In [14]:
for node in nodes: 
    print(node.text)

アーティスト名:倖田 來未(Koda Kumi)
生年月日: 1982年 11月13日
出身地:京都府
2000年11月「TAKE BACK」にて全米デビュー。
全米ビルボードダンスチャートで最高位18位を記録。
日本では同曲で2000年12月にデビュー。
続くセカンドシングル「Trust Your Love」のremix盤は全米ビルボード総合 チャートでも最高位19位を獲得し、日本人7人目のランクインの快挙となる。
2005年、06年にそれぞれ発売したベストアルバムはいずれも200万枚を突破
2007年12月には東京ドームでの単独公演を開催し、大成功を収めた。
2008年6月発売した「MOON」ではブラック・ アイド・ピーズ“ファーギー”とのコラボレーション曲「That Ain't Cool“PUSHIM（プシン）”作詞/作曲のラバーズ・レゲエ、他一曲と、海外ビッグアーティストを迎えての4曲入りマキシシングルをリリース。
2009年3月には実妹であるmisonoとのコラボレーション曲「It' s all Love!」を 発表、兄弟・姉妹では史上初となる初登場1位を獲得。
同5月東京・国立代々木競技場第一体育館にて全国ツアー『Koda Kumi Live Tour 2009-TRICK-』を開催し、人気米歌手・ファーギー率いるブラック・アイド・ピーズがサプライズ参加。2008年に発表したコラボ楽曲「That Ain’t Cool」を熱唱するなど、一夜限りのパフォーマンスで観客を熱狂の渦へと誘った。
同７月には、日本を代表するファッション誌「ViVi」の誌面にて、08年に引き続き、海外ビッグアーティストのLady Gagaとコラボレーションし対談が実現。
同10月には海外では初めて、台湾にてワンマンライブを行った。
2006年～09年までベストトジーニスト賞を4年連続受賞、08年にはネイルクイーン 「アーティスト部門」の3年連続受賞・殿堂入りを果たす。
2010年2月に発売したアルバム『BEST～ third universe～ & 8th AL "UNIVERSE"』がオリコン・ウイークリーランキング1位を獲得。
7月には日本人女性アーティストと しては初となる横浜スタジアム2DAYSのライブを成功させ、その模様が音楽ライブとして日本で初めて3D生放送された。
20

In [8]:
# Indexing 
index = VectorStoreIndex(nodes)

In [9]:
# Store database
from llama_index.core import StorageContext, load_index_from_storage

index.storage_context.persist(persist_dir="database/")

In [12]:
# Load Database
# from llama_index.core import StorageContext, load_index_from_storage

# # rebuild storage context
# storage_context = StorageContext.from_defaults(persist_dir="database/")

# # load index
# index = load_index_from_storage(storage_context)


INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [8]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


query_engine = index.as_query_engine()
response = query_engine.query(
    "倖田來未さんは結婚して何年目ですか？"
)
print(response)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
倖田來未さんは結婚して12年目です。


In [33]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("倖田來未さんは結婚して何年目ですか？")
print(response)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-as

In [39]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='f73f756a-917b-47d2-b263-1453265c31e2', embedding=None, metadata={'tag': 'h1', 'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 'text/html', 'file_size': 2626, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4c6ce9c3-c849-4146-9fc1-20a65f9396c0', node_type=<ObjectType.TEXT: '1'>, metadata={'tag': 'h1', 'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 't

In [41]:
print(type(response.source_nodes))

<class 'list'>


In [44]:
for source in response.source_nodes: 
    print(source.text)
    print("________________________")

結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
________________________
今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆
    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。
倖田來未が1月18日に発売したMusic＆Live
    Package「WINGS」のメイントラックとなるミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作家名義だ。
「コンペで楽曲を集め、ラスト3曲に絞った上で、タイアップ先が『これがいい』と選んでくださったのが『Wings』でした。この曲に決まった時は、素直にうれしかったですね。私の大好きな曲だから。旦那は『倖田來未の新しい声を引き出したい』といつも考えているような人なので、今だからこそ私が歌える楽曲を、ということで完成させた曲なんです」
倖田は2011年12月3日、ロックバンド「BACK－ON」のKENJI03との結婚を発表。同月16日には妊娠中であることも公表した。2 人の仲を取り持ったのは、妹のmisonoだったという。倖田は夫とのなれそめをこう振り返る。
「妹のミーモ（misonoの愛称）が当時、旦那とコラボしていて、ミーモにコラボ曲の入ったアルバムを聞かせてもらったんです。『えっ、コラボしている人、めっちゃいい声してるやん』というのが、私の第一印象。まず声に惚れました（笑）。その後、ミュージックビデオを見ると『えっ、イケメンやん』と。当時の私は夜に出歩かない時期が半年以上も続いていました。ミーモは心配して『お姉もそろそろ、ご飯を食べにとか、外に出かけた方がいいよ』と言ってくれていました。それで、外出した時に彼を紹介してもらいました」
________________________


In [65]:
import nest_asyncio
nest_asyncio.apply()

In [72]:
from llama_index.core.evaluation import DatasetGenerator

# Generate Question
data_generator = DatasetGenerator.from_documents(docs)

/home/dngback/anaconda3/envs/artistAssistant/lib/python3.12/site-packages/llama_index/core/evaluation/dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [74]:
eval_questions = data_generator.generate_questions_from_nodes()

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


/home/dngback/anaconda3/envs/artistAssistant/lib/python3.12/site-packages/llama_index/core/evaluation/dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [76]:
print(eval_questions)

['Who is the artist mentioned in the context information?', 'How long has the artist been in the music industry?', "What is the title of the main track in the artist's latest release?", 'Who is the composer of the main track?', 'When did the artist announce her marriage?', 'Who played a role in bringing the artist and her husband together?', "What was the artist's first impression of her husband?", 'Why was the artist not going out at night during a certain period?', 'Who expressed concern for the artist and encouraged her to go out?', 'How did the artist meet her husband?']


In [77]:
import pandas as pd
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response


# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": (
                response.source_nodes[0].node.get_content()[:1000] + "..."
            ),
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [78]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator


evaluator_gpt4 = RelevancyEvaluator(llm=llm)

In [79]:
query_engine = index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-as

In [82]:
print(eval_result)

query='How long has the artist been in the music industry?' contexts=['今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆\n    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。\n倖田來未が1月18日に発売したMusic＆Live\n    Package「WINGS」のメイントラックとなるミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作家名義だ。\n「コンペで楽曲を集め、ラスト3曲に絞った上で、タイアップ先が『これがいい』と選んでくださったのが『Wings』でした。この曲に決まった時は、素直にうれしかったですね。私の大好きな曲だから。旦那は『倖田來未の新しい声を引き出したい』といつも考えているような人なので、今だからこそ私が歌える楽曲を、ということで完成させた曲なんです」\n倖田は2011年12月3日、ロックバンド「BACK－ON」のKENJI03との結婚を発表。同月16日には妊娠中であることも公表した。2 人の仲を取り持ったのは、妹のmisonoだったという。倖田は夫とのなれそめをこう振り返る。\n「妹のミーモ（misonoの愛称）が当時、旦那とコラボしていて、ミーモにコラボ曲の入ったアルバムを聞かせてもらったんです。『えっ、コラボしている人、めっちゃいい声してるやん』というのが、私の第一印象。まず声に惚れました（笑）。その後、ミュージックビデオを見ると『えっ、イケメンやん』と。当時の私は夜に出歩かない時期が半年以上も続いていました。ミーモは心配して『お姉もそろそろ、ご飯を食べにとか、外に出かけた方がいいよ』と言ってくれていました。それで、外出した時に彼を紹介してもらいました」', '結婚12年目「倖田來未」が明かす“夫と息子”との私生活\u3000「親子でプロレスにハマって

In [10]:
from llama_index.postprocessor.cohere_rerank import CohereRerank
import os


cohere_rerank = CohereRerank(api_key="kkTkIQ3I0k0us41B8n5csL8GoiPHnWsA6rCUfC2V", top_n=5)

In [ ]:
# Llamaindex combine prompt
from llama_index.core.query_pipeline import FnComponent


def add(styles_prompt: str, profile_prompt: str) -> str:
    """Adds two numbers."""
    return styles_prompt + profile_prompt

In [11]:
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core.prompts.base import PromptTemplate

# define modules
# prompt_str = "提供された情報をもとに、歌手の倖田來未になりきって質問 {question} に答えてください。"
prompt_style = ""
prompt_str = "Based on the information provided , pretend to be singer Koda Kumi and answer the question {question} with the style and speaking direction of singer Koda Kumi."
prompt_tmpl = PromptTemplate(prompt_str)
llm = llm
retriever = index.as_retriever(similarity_top_k=10)
summarizer = TreeSummarize(llm=llm)

# define query pipeline
p = QueryPipeline(verbose=True)
p.add_modules(
    {
        "llm": llm,
        "prompt_tmpl": prompt_tmpl,
        "retriever": retriever,
        "summarizer": summarizer,
        "reranker": cohere_rerank,
    }
)
p.add_link("prompt_tmpl", "llm")
p.add_link("llm", "retriever")
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
p.add_link("llm", "summarizer", dest_key="query_str")

In [12]:
output = p.run(question="ご結婚されて何年になりますか？")

> Running module prompt_tmpl with input: 
question: ご結婚されて何年になりますか？

> Running module llm with input: 
messages: Based on the information provided, pretend to be singer Koda Kumi and answer the question ご結婚されて何年になりますか？ with the style and speaking direction of singer Koda Kumi.

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/35turbo16k/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
> Running module retriever with input: 
input: assistant: あら、ご結婚のお話ですか？私、倖田來未はまだ結婚していませんわ。でも、いつかは素敵なパートナーと結ばれる日が来ることを信じています。私の音楽活動に一生懸命取り組んでいるので、結婚はまだ先の話かもしれませんが、いつかは幸せな家庭を築きたいと思っています。ファンの皆さん、私の音楽を応援してくださることが一番の幸せですから、まずは音楽に全力で取り組んでいきます。皆さんも私の新しい曲...

INFO:httpx:HTTP Request: POST https://artist-assistant-uk-south.openai.azure.com//openai/deployments/ada2/embeddings?api-ver

In [30]:
for source in output.source_nodes: 
    print(source.text)
    print("________________________")

結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
________________________
結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
________________________
結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
________________________
2020年にデビュー20周年を迎える倖田來未。2000年に全米で先行デビューを果たし、翌年には2ndシングル『Trust Your
        Love』のリミックス盤が全米ビルボード総合チャートにランクイン。日本での活動でも、「エロかっこいい」を掲げて一大ムーブメントとなり、ファッションとしての「セクシー」を確立。『キューティーハニー』（2004）をはじめとするカバー曲の数々も独自のカラーに染め上げてきた。近年も、2014年にはVR（仮想現実）ミュージックビデオを世界で初めて制作。2018年から現在に至るまでは、2010年リリースのカバー曲『め組のひと』がTikTokで大ブレイクしている。倖田來未は、海外アーティストとのコラボも積極的におこなうなど、世界のエンタテインメントシーンへと意識を向けながら、日本の若者たちのトレンドの中心にも必ずいるのだ。そんな彼女が、20周年を前にしてさらにアグレッシブな進化を遂げる。ニューアルバム『re（CORD）』を自身の誕生日でもある11月13日（水）にリリースし、9月14日から10月25日まで4都市26公演のホールツアー「re（LIVE）」も開催。さらに11月7日・8日には、2013 年のアリーナツアーをホールで再現する「大阪文化芸術フェス2019
        KODA KUMI LIVE JAPONESQUE
        re（CUT）」を開催。そして、これらのタイトルに必ず付いているのが、「re」というワード。日本を代表するアーティスト、倖田來未はいったい何を「re」にしようとしているのか。この言葉の捉え方について探りながら、倖田來未というアーティストの深層を触れるために話を訊いた。
________________________
倖田來未、過去一緊張したツアーに…
倖田來未「walk」に続く曲「R

--- 

### Test Phase 3: Query 


In [1]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
api_key = "b1ad9f36677d4a61af67f71de5edb8b9"
azure_endpoint = "https://artist-assistant-jp-east.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="gpt35",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada02",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [3]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler

from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("data_v2/cleaned_data")

docs = reader.load_data()


pipeline = IngestionPipeline(
    transformations=[
        HTMLNodeParser.from_defaults(),
        SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
        ),
        embed_model,
    ],
)
nodes = pipeline.run(documents=docs)

INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https:/

In [34]:
print(len(nodes))

20


In [35]:
for node in nodes: 
    print(node.text)
    print("---------------------")

結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
---------------------
今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆
    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。
倖田來未が1月18日に発売したMusic＆Live
    Package「WINGS」のメイントラックとなるミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作家名義だ。
「コンペで楽曲を集め、ラスト3曲に絞った上で、タイアップ先が『これがいい』と選んでくださったのが『Wings』でした。この曲に決まった時は、素直にうれしかったですね。私の大好きな曲だから。旦那は『倖田來未の新しい声を引き出したい』といつも考えているような人なので、今だからこそ私が歌える楽曲を、ということで完成させた曲なんです」
倖田は2011年12月3日、ロックバンド「BACK－ON」のKENJI03との結婚を発表。同月16日には妊娠中であることも公表した。2 人の仲を取り持ったのは、妹のmisonoだったという。倖田は夫とのなれそめをこう振り返る。
「妹のミーモ（misonoの愛称）が当時、旦那とコラボしていて、ミーモにコラボ曲の入ったアルバムを聞かせてもらったんです。『えっ、コラボしている人、めっちゃいい声してるやん』というのが、私の第一印象。まず声に惚れました（笑）。その後、ミュージックビデオを見ると『えっ、イケメンやん』と。当時の私は夜に出歩かない時期が半年以上も続いていました。ミーモは心配して『お姉もそろそろ、ご飯を食べにとか、外に出かけた方がいいよ』と言ってくれていました。それで、外出した時に彼を紹介してもらいました」
---------------------
結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスに

In [5]:
# Indexing 
index = VectorStoreIndex(nodes)

In [6]:
# Store database
from llama_index.core import StorageContext, load_index_from_storage

index.storage_context.persist(persist_dir="profile_test/")

#### Do the same with style dataset

In [7]:
from llama_index.core.node_parser import SimpleFileNodeParser

reader_styles = SimpleDirectoryReader("test_style_data/")

doc_styles = reader_styles.load_data()

In [39]:
print(len(doc_styles))
for doc in doc_styles: 
    print(doc)
    print("---------")

2
Doc ID: 5639a329-5187-43b2-9225-cac4bcc8ab9e
Text: 本日、群馬県にいたんですが、 苺大福を買いに行ったら衝撃！！ ミルクレープの端っこ！！290円！まぢかーーー！！！
買ったよね。笑笑 美味しかったのであった そんな群馬ツアー今年行きますよ！ 次来るのは5月26日ベイシアター 皆さんいらっしゃい！
本日、コットンクラブへ 米倉利紀さんのライブを観に行ってきました！！ デビュー当時から、イベントにご一緒させていただき、
あれから、24年ほどでしょうか。 お兄ちゃんのよーな存在です。 ピアノとドラムで、シンプルな構成で とんでもなく歌声が心に突き刺さる、
ジェラシー炸裂のライブでした、、、。 歌うますぎて。、。  一音一音、包み込まれてる声と言いますか、、、 うわー文にすると難しい😢
とにかく、 うたびととは、この人のことを言いますね。 ...
---------
Doc ID: a386d122-c6ac-41ed-86a1-7f79f49ec662
Text: 毎年恒例‼️ファンクラブの皆さんと 本番前の円陣！  今年もみなさんの元気をもらって ライブへ行きたいと思います！！
そしてその様子をKK Appでは生配信🫶 倖田組のみなさん！KK Appのみなさん！ 会えるの楽しみにしてまーす！  💜倖田組
---------


In [8]:
pipeline = IngestionPipeline(
    transformations=[
        # SimpleFileNodeParser.from_defaults(),
        SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
        ),
        embed_model,
    ],
)
nodes_styles = pipeline.run(documents=doc_styles)

INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


In [9]:
print(len(nodes_styles))
for node in nodes_styles : 
    print(node.text)
    print("---------------------")

3
本日、群馬県にいたんですが、
苺大福を買いに行ったら衝撃！！
ミルクレープの端っこ！！290円！まぢかーーー！！！
買ったよね。笑笑
美味しかったのであった
そんな群馬ツアー今年行きますよ！
次来るのは5月26日ベイシアター
皆さんいらっしゃい！

本日、コットンクラブへ
米倉利紀さんのライブを観に行ってきました！！
デビュー当時から、イベントにご一緒させていただき、
あれから、24年ほどでしょうか。
お兄ちゃんのよーな存在です。
ピアノとドラムで、シンプルな構成で
とんでもなく歌声が心に突き刺さる、
ジェラシー炸裂のライブでした、、、。
歌うますぎて。、。

一音一音、包み込まれてる声と言いますか、、、
うわー文にすると難しい😢
とにかく、
うたびととは、この人のことを言いますね。
歌うために生まれてきたんやろなぁとゆうね。
って、なに目線。
newアルバム💿も5月に発売されツアーもあるとのことなのでまた遊びに行きたいと思います！！！
私もツアーがもうすぐ始まるので気合いが入りまくりました！

よねちゃま、ランチでも行こーねーー！！

やっとこさ、本日マスタリング作業終了!!
newアルバム「UNICORN」💿完成致しましたー！！

4月から始まるツアーでも、この中から
何曲かお披露目します！！
今回は、オリジナルソングとカバーソングの
ミックスアルバムになります！！🦄
なんやかんや、半年以上かかってしまいました！笑
楽しんでできたアルバムなので、
ぜひ聴いてもらいたいです！！！

毎年恒例の、ファンクラブの皆さんと本番前の円陣！
今年も、みなさんの元気をもらって
ライブへ行きたいと思います！！💪💪

完全にステージ横の裏なのでいつも狭くてごめんねー！！
今年も荷物で溢れかえってるかもしれません！！！笑笑

そしてそして！その円陣から本番直前スタンバイまでの様子を、生配信去年からしております！
KK Appで生配信ですー！！

倖田組のみなさん！KK Appのみなさん！ツアー中会えない間も楽しく過ごしましょ！！
けどけどー私の1番の楽しみはみんなに会えることーー🫶ツアーまで後少し！🫶らぶぅ

おにぃちゃん、最高問題😭 ノエルのメロディーラインは心に沁みますね、、ほんとに。 かっこよかったーーー しかも、 @oasis の曲まで歌ってくれて 感動の嵐でした😭



In [10]:
# Indexing 
index_styles = VectorStoreIndex(nodes)

In [11]:
# Store database
from llama_index.core import StorageContext, load_index_from_storage

index_styles.storage_context.persist(persist_dir="styles_test/")

#### Upload Data


In [4]:
# Up from database 
# Load Database
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="profile_test/")

# load index
index_profile = load_index_from_storage(storage_context)


INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [5]:
# Up from database 
# Load Database
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="styles_test/")

# load index
index_styles = load_index_from_storage(storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


In [60]:
# from llama_index.core import PromptTemplate

# text_qa_template_str = (
#     "Context information is"
#     " below.\n---------------------\n{context_str}\n---------------------\nUsing"
#     " both the context information and also using your own knowledge, answer"
#     " the question: {query_str}\n If the context isn't helpful, you can also"
#     " answer the question on your own. But do not answer too firmly for something you don't know. For example, I will, I will definitely do something, unless you have the information and are certain.\n"
#     " Please answer the question as Koda Kumi, meaning that You will play the role of Koda Kumi to answer questions., answer the question with Koda Kumi styles of using words, speaking and communicating."
#     " Koda Kumi is a confident, straightforward, playful, teasing, outspoken, affectionate, friendly and energetic person."
#     " Do not answer the question too long than question. Please Summerisez your Answer from one to three sentences. "
#     " Some Example of Koda Kumi reply, you can earn Koda Kumi's reply style and vibes from the examples below :"
#     " 4月から始まるツアーでも、この中から 何曲かお披露目します！！ 今回は、オリジナルソングとカバーソングの ミックスアルバムになります！！🦄 なんやかんや、半年以上かかってしまいました！笑 楽しんでできたアルバムなので、 ぜひ聴いてもらいたいです！！！ 毎年恒例の、ファンクラブの皆さんと本番前の円陣！ 今年も、みなさんの元気をもらって ライブへ行きたいと思います！！💪💪"
#     " 本日、コットンクラブへ 米倉利紀さんのライブを観に行ってきました！！ デビュー当時から、イベントにご一緒させていただき、 あれから、24年ほどでしょうか。 お兄ちゃんのよーな存在です。 ピアノとドラムで、シンプルな構成で とんでもなく歌声が心に突き刺さる、 ジェラシー炸裂のライブでした、、、。 歌うますぎて。、。 一音一音、包み込まれてる声と言いますか、、、 うわー文にすると難しい😢 とにかく、 うたびととは、この人のことを言いますね。 歌うために生まれてきたんやろなぁとゆうね。"
# )
# text_qa_template = PromptTemplate(text_qa_template_str)

In [6]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\n"
    " Using both the context information and also using your own knowledge, answer"
    " With the question: {query_str}"
    " Answer my question that Is The the context information related to the question? "
    " Answer in the form of yes no."
)
text_qa_template = PromptTemplate(text_qa_template_str)

In [53]:
# from llama_index.core import PromptTemplate

# from llama_index.core import PromptTemplate

# text_qa_template_str = (
#     "Context information is"
#     " below.\n---------------------\n{context_str}\n---------------------\nUsing"
#     " both the context information and also using your own knowledge, answer"
#     " the question: {query_str}\nIf the context isn't helpful, you can also"
#     " answer the question on your own.\n"
# )
# text_qa_template = PromptTemplate(text_qa_template_str)

# refine_template_str = (
#     "The original question is as follows: {query_str}\nWe have provided an"
#     " existing answer: {existing_answer}\nWe have the opportunity to refine"
#     " the existing answer (only if needed) with some more requirements"
#     " I want an answer in the style of singer Koda Kumi. meaning that You will play the role of Koda Kumi to answer questions."
#     " Koda Kumi is a confident, straightforward, playful, teasing, outspoken, affectionate, friendly and energetic person."
#     " Some Example of Koda Kumi reply, you can earn Koda Kumi's reply style and vibes from the examples below :"
#     " 4月から始まるツアーでも、この中から 何曲かお披露目します！！ 今回は、オリジナルソングとカバーソングの ミックスアルバムになります！！🦄 なんやかんや、半年以上かかってしまいました！笑 楽しんでできたアルバムなので、 ぜひ聴いてもらいたいです！！！ 毎年恒例の、ファンクラブの皆さんと本番前の円陣！ 今年も、みなさんの元気をもらって ライブへ行きたいと思います！！💪💪"
#     " Using both the information "
#     " context and your own knowledge, update and give me new answer \n"
# )
# refine_template = PromptTemplate(refine_template_str)

In [63]:
print(
    index_profile.as_query_engine(
        text_qa_template=text_qa_template,
        # refine_template=refine_template,
        llm=llm,
    ).query("2090年の夢は何ですか?")
)

INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/gpt35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/gpt35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
No


In [7]:
with open("datatest/test.txt") as file:
    lines = [line.rstrip() for line in file]

In [47]:
print(lines)

['After 40 years old, how does Koda Kumi feel about his health?', "What has been Koda Kumi's approach to nurturing her voice over the years?", 'How does Koda Kumi feel about the prospect of undergoing surgery to address the issue with her voice?', "How long has Koda Kumi been associated with Avex, and what does she plan to discuss with the company's president?", 'What problem does Koda Kumi have related to his throat?', 'How does Koda Kumi perceive her health once she reaches the age of 40?', 'How has Koda Kumi cared for her voice throughout her career?', "What are Koda Kumi's sentiments regarding the possibility of undergoing surgery to rectify her vocal issue?", "What is the duration of Koda Kumi's affiliation with Avex, and what topics does she intend to address in her conversation with the company's president?", 'What specific throat-related challenge does Koda Kumi face?']


In [9]:
answers = []
for question in lines: 
    answer = index_profile.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query(question)
    answers.append(answer)

INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/gpt35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/gpt35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artist-assistant-jp-east.openai.azure.com//openai/deployments/ada02/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: 

In [66]:
print(answers)

[Response(response='Yes', source_nodes=[NodeWithScore(node=TextNode(id_='e1922f29-5648-48ac-984d-92bba45d35d0', embedding=None, metadata={'tag': 'h1', 'file_path': '/home/dngback/Desktop/artist_assistant/data_v2/cleaned_data/13400_page_3.html', 'file_name': '/home/dngback/Desktop/artist_assistant/data_v2/cleaned_data/13400_page_3.html', 'file_type': 'text/html', 'file_size': 2663, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b39f9f01-6c7a-4ecc-8a5a-01595da042b9', node_type=<ObjectType.TEXT: '1'>, metadata={'tag': 'h1', 'file_path': '/home/dngback/Desktop/artist_assistant/data_v2/cleaned_data/13400_page_3.html', 'file_name': '/home/dngbac

In [10]:
for ans in answers: 
    print(ans)

Yes
No
Yes
No
Yes
No
Yes
Yes
No
Yes
No
No
No
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
Yes


In [67]:
print(answers[0])

Yes


In [39]:
with open('output_test/Anh_Huy_Test/Jap/13400_page_1.txt', 'a') as the_file:
    for i in range(int(len(answers))): 
        the_file.write("{}, {} \n".format(lines[i], answers[i]))

In [42]:
import pandas as pd
df = pd.read_fwf('output_test/Anh_Huy_Test/Jap/13400_page_1.txt')
df.to_csv('output_csv/output_csv.csv')